In [11]:
import pandas as pd
import numpy as np
import string
import sqlite3
import re


from flask import Flask, request, jsonify
from nltk.tokenize import RegexpTokenizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords

In [2]:
conn = sqlite3.connect('goldlevel/goldchallenge.db')
cursor = conn.cursor()

In [3]:
kamusalay = '''CREATE TABLE kamusalay(
                column1 TEXT,
                column2 TEXT)'''

cursor.execute(kamusalay)
conn. commit()

print("create table successful")

create table successful


In [4]:
entities = pd.read_csv('new_kamusalay.csv', encoding='Latin-1')
    
for index, row in entities.iterrows():
    cursor.execute("INSERT INTO kamusalay VALUES(?,?)", tuple(row))
    conn.commit()
    
print("create table successful")

create table successful


In [7]:
abusive = '''CREATE TABLE abusive(
                column1 TEXT)'''

cursor.execute(abusive)
conn. commit()

print("create table successful")

create table successful


In [ ]:
entities = pd.read_csv('abusive.csv', encoding='Latin-1')
    
for index, row in entities.iterrows():
    cursor.execute("INSERT INTO abusive VALUES(?)", tuple(row))
    conn.commit()
    
print("create table successful")

In [38]:
cleaningtext = '''CREATE TABLE cleaningtext(
                        cleaningtext_hasil TEXT)'''

cursor.execute(cleaningtext)
conn. commit()

print("create table successful")

create table successful


In [40]:
app = Flask(__name__)

from flask import request
from flasgger import Swagger, LazyString, LazyJSONEncoder
from flasgger import swag_from

app.json_encoder = LazyJSONEncoder
swagger_template = dict(
info = {
    'title': LazyString(lambda: 'Data Science Challenge'),
    'version': LazyString(lambda: '1.0.0'),
    'description': LazyString(lambda: 'Gold Challenge'),
    },
    host = LazyString(lambda: request.host)
)

swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": 'docs',
            "route": '/docs.json',
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}


swagger = Swagger(app, template=swagger_template,             
                  config=swagger_config)

@swag_from("/Users/bianda/Desktop/DSC/API-Binar/docs/hello_world.yml", methods=['GET'])
@app.route('/', methods=['GET'])
def hello_world():
    json_response = {
        'status_code': 200,
        'description': "Menyapa Hello World",
        'data': "Hello World",
    }

    response_data = jsonify(json_response)
    return response_data

@swag_from("/Users/bianda/Desktop/DSC/API-Binar/docs/text.yml", methods=['GET'])
@app.route('/text', methods=['GET'])
def text():
    json_response = {
        'status_code': 200,
        'description': "Original Teks",
        'data': "Halo, apa kabar semua?",
    }

    response_data = jsonify(json_response)
    return response_data

@swag_from("/Users/bianda/Desktop/DSC/API-Binar/docs/text_processing.yml", methods=['POST'])
@app.route('/text-processing', methods=['POST'])
def text_processing():

    text = request.form.get('text')
    
    #mengubah ke huruf kecil
    cleaned_text_lower = text.lower()
    
    #remove RT
    cleaned_text_rt = re.sub(r'^rt[\s]+', ' ', cleaned_text_lower)
    
    #remove user
    cleaned_text_user = re.sub(r'^user[\s]+', '', cleaned_text_rt)
    
    #remove url
    cleaned_text_url = re.sub(r'<.*?>', r' ', cleaned_text_user)
    
    #remove tandabaca
    cleaned_text_punct = re.sub(r'[^\w\s\d]', ' ', cleaned_text_url)
    
    #remove dem
    cleaned_text_dem = re.sub(r'[\\x]+[a-z0-9]{2}', '', cleaned_text_punct)
    
    #remove number
    cleaned_text_num = re.sub(r'\d+', ' ', cleaned_text_dem)
    
    # Mengganti karakter selain huruf dan angka dengan spasi
    cleaned_text_alpha = re.sub(r'[^a-zA-Z]', ' ', cleaned_text_num)
    
    # Menghilangkan white space tambahan 
    # .split() = memisahkan teks menjadi spasi, ''.join() = menggabungkan potongan
    cleaned_text_final = ' '.join(cleaned_text_alpha.split())
    
    def read_replacing_word(cursor):
        cursor.execute("SELECT column1, column2 FROM kamusalay")
        rows = cursor.fetchall()
        replacing_word_dict = {}
        for row in rows:
            replacing_word_dict[row[0]] = row[1]
        return replacing_word_dict

    def replacing_term(replaceword, replacing_word_dict):
        return [replacing_word_dict[term] if term in replacing_word_dict else term for term in replaceword]
    
    conn = sqlite3.connect('goldlevel/goldchallenge.db')
    cursor = conn.cursor()
        
    replacing_word_dict = read_replacing_word(cursor)
    replaced_text = replacing_term(cleaned_text_final.split(), replacing_word_dict)
    
    def read_abusive_word(cursors):
        cursors.execute("SELECT column1 FROM abusive")
        rows = cursors.fetchall()
        abusive_word = [row[0] for row in rows]
        return abusive_word
    
    def remove_abusive(replaced_text, abusive_word):
        words = [word for word in replaced_text if word.lower() not in abusive_word]
        return words
    
    conn = sqlite3.connect('goldlevel/goldchallenge.db')
    cursors = conn.cursor()
        
    abusive_word = read_abusive_word(cursors)

    stopword_remove = remove_abusive(replaced_text, abusive_word)
    print("stopword_remove:", stopword_remove)
    
    json_response = {
        'status_code': 200,
        'description': "Teks yang sudah diproses",
        'data': ' '.join(stopword_remove),  
    }

    response_data = jsonify(json_response)
    return response_data

@swag_from("/Users/bianda/Desktop/DSC/API-Binar/docs/text_processing_file.yml", methods=['POST'])
@app.route('/text-processing-file', methods=['POST'])
def text_processing_file():

    # Upladed file
    file = request.files.getlist('file')[0]

    # Import file csv ke Pandas
    df = pd.read_csv(file, encoding='latin-1')

    # Ambil teks yang akan diproses dalam format list
    texts = df['Tweet'].to_list()

    # Lakukan cleansing pada teks
    cleaned_text = []
    for text in texts:
         
        #mengubah ke huruf kecil
        text_lower = text.lower()    
            
        #remove RT
        text_rt = re.sub(r'^rt[\s]+', ' ', text_lower)
    
        #remove user
        text_user = re.sub(r'^user[\s]+', '', text_rt)
    
        #remove url
        text_url = re.sub(r'<.*?>', r' ', text_user)
    
        #remove tandabaca
        text_punct = re.sub(r'[^\w\s\d]', ' ', text_url)
    
        #remove dem
        text_dem = re.sub(r'[\\x]+[a-z0-9]{2}', '', text_punct)
    
        #remove number
        text_num = re.sub(r'\d+', ' ', text_dem)
        
        # Remove hashtag
        text_hashtag = re.sub(r'#\w+', ' ', text_num)
    
        # Mengganti karakter selain huruf dan angka dengan spasi
        text_alpha = re.sub(r'[^a-zA-Z]', ' ', text_hashtag)
          
        #remove white space
        text_space = ' '.join(text_alpha.split())
        
        def read_replacing_word(cursor):
            cursor.execute("SELECT column1, column2 FROM kamusalay")
            rows = cursor.fetchall()
            replacing_word_dict = {}
            for row in rows:
                replacing_word_dict[row[0]] = row[1]
            return replacing_word_dict

        def replacing_term(replaceword, replacing_word_dict):
            return [replacing_word_dict[term] if term in replacing_word_dict else term for term in replaceword]
    
        conn = sqlite3.connect('goldlevel/goldchallenge.db')
        cursor = conn.cursor()
        
        replacing_word_dict = read_replacing_word(cursor)
        replaced_text = replacing_term(text_space.split(), replacing_word_dict)
         
        def read_abusive_word(cursors):
            cursors.execute("SELECT column1 FROM abusive")
            rows = cursors.fetchall()
            abusive_word = [row[0] for row in rows]
            return abusive_word
    
        def remove_abusive(replaced_text, abusive_word):
            words = [word for word in replaced_text if word.lower() not in abusive_word]
            return words
    
        conn = sqlite3.connect('goldlevel/goldchallenge.db')
        cursors = conn.cursor()
        
        abusive_word = read_abusive_word(cursors)

        abusive_remove = remove_abusive(replaced_text, abusive_word)
        print("abusive_remove:", abusive_remove)    
        
        cleaned_text.append(' '.join(map(str, abusive_remove)))
        
        df = pd.DataFrame({'cleaned_text': cleaned_text})
        
        conn = sqlite3.connect('goldlevel/goldchallenge.db')
        df.to_sql('cleaningtext', conn, if_exists='replace', index=False)
    
    json_response = {
        'status_code': 200,
        'description': "Teks yang sudah diproses",
        'data': (cleaned_text),
    }

    response_data = jsonify(json_response)
    return response_data

if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Dec/2023 01:56:43] "GET /docs/ HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 01:56:43] "GET /docs.json HTTP/1.1" 200 -


abusive_remove: ['di saat', 'semua', 'cowok', 'berusaha', 'melacak', 'perhatian', 'gue', 'kamu', 'lantas', 'remehkan', 'perhatian', 'yang', 'gue', 'kasih', 'khusus', 'ke', 'kamu', 'basic', 'kamu', 'cowok']
abusive_remove: ['pengguna', 'pengguna', 'siapa', 'yang', 'telat', 'memberi', 'tau', 'kamu', 'gue', 'bergaul', 'dengan', 'cigax', 'jifla', 'calis', 'sama', 'siapa', 'itu', 'licew', 'juga']
abusive_remove: ['kadang', 'aku', 'berpikir', 'kenapa', 'aku', 'tetap', 'percaya', 'pada', 'tuhan', 'padahal', 'aku', 'selalu', 'jatuh', 'berkali', 'kali', 'kadang', 'aku', 'merasa', 'tuhan', 'itu', 'meninggalkan', 'aku', 'sendirian', 'ketika', 'orang tuaku', 'berencana', 'berpisah', 'ketika', 'kakakku', 'lebih', 'memilih', 'jadi', 'kristen', 'ketika', 'aku', 'anak', 'ter']
abusive_remove: ['pengguna', 'aku', 'itu', 'aku', 'dan', 'ku', 'tau', 'matamu', 'tapi', 'dilihat', 'dari', 'mana', 'itu', 'aku']
abusive_remove: ['pengguna', 'kaum', 'sudah', 'kelihatan', 'dongoknya', 'dari', 'awal', 'tambah', '

abusive_remove: ['a', 'orang', 'yang', 'suka', 'ke', 'konser', 'konser', 'sedikit', 'boros']
abusive_remove: ['sampai', 'kapan', 'makan', 'racun', 'banyak', 'ulama', 'yang', 'mengatakan', 'hukum', 'rokok', 'mubah', 'dan', 'ada', 'juga']
abusive_remove: ['susilo', 'bambang', 'yudhoyono', 'presiden', 'ri', 'ke', 'meminta', 'khofifah', 'emil', 'untuk', 'dekat', 'dengan', 'masyarakat', 'no', 'bisa', 'bagus', 'khofifahemilnomer']
abusive_remove: ['sudah', 'biasa', 'kali', 'gue', 'penggemar korea pop', 'tapi', 'kalau', 'mengomong', 'sama', 'teman', 'gue', 'idol', 'gue']
abusive_remove: ['saya', 'suka', 'video', 'pengguna', 'uniform resource locator', 'awal', 'kisah', 'perseteruan', 'haji', 'bolot', 'amp', 'malih', 'tongtong']
abusive_remove: ['budaya', 'bersungguh', 'kalau', 'hal', 'islam', 'tidak', 'diindahkan']
abusive_remove: ['pengguna', 'pengguna', 'pengguna', 'lah', 'lah', 'lah', 'hem', 'hem', 'semua', 'diam', 'semua', 'dan', 'kita', 'coba', 'tanya', 'sama', 'rumput', 'yang', 'bergoyan

abusive_remove: ['pengguna', 'mengakuinya', 'orang', 'pintar', 'dan', 'beragama', 'memangnya', 'agama', 'mana', 'yang', 'memperbolehkan', 'menista', 'dan', 'menghujat', 'orang', 'lain', 'jangankan', 'orang', 'orang', 'yang', 'maaf', 'sejak', 'lahir', 'sekalipun', 'dia', 'kita', 'harus', 'hargai', 'sebagai', 'sesama', 'ciptaan', 'tuhan', 'ingat', 'itu', 'juga', 'pintar', 'dan', 'sangat', 'licik']
abusive_remove: ['pendaftaran', 'calon presiden', 'untuk', 'pemilihan presiden', 'saja', 'belum', 'dibuka', 'sampai', 'agustus', 'yang', 'normatif', 'begini', 'politikus', 'partai demokrasi indonesia perjuangan', 'masih', 'belum', 'paham', 'ganti presiden']
abusive_remove: ['sih', 'sudah', 'tau', 'pemerintah', 'ini', 'bobrok', 'koruptor', 'bisa', 'pimpinan', 'sidang', 'dewan perwakilan rakyat', 'paya', 'uniform resource locator']
abusive_remove: ['dari', 'dulu', 'kenapa', 'bapak', 'gue', 'kalau', 'ke', 'mantan', 'mantan', 'gue', 'selalu', 'merasa', 'sedih', 'sampai', 'kepikiran', 'dan', 'tidak 

...

abusive_remove: ['setiap', 'mau', 'survei', 'ke', 'pulau', 'terpencil', 'di', 'nabire', 'papua', 'pasti', 'tidak', 'pernah', 'lupa', 'bertanya', 'ada', 'muaranya', 'tidak', 'dan', 'jawabannya', 'pasti', 'ada', 'di', 'situ', 'gue', 'berharap', 'kebesaran', 'allah', 'untuk', 'menjaga', 'keselamatan', 'semuanya']
abusive_remove: ['allah', 'hu', 'barbar', 'memang', 'biadab', 'bang', 'fahri', 'muslim', 'tak', 'mungkin', 'hidup', 'tanpa', 'anda', 'pakai', 'internet', 'kan']
abusive_remove: ['subhan', 'allah', 'kenangan', 'yang', 'tidak', 'akan', 'terlupakan', 'saat', 'umat', 'islam', 'bersatu', 'melaksanakan', 'salat', 'jum', 'at', 'pada', 'aksi', 'bulan', 'terlewat', 'padahal', 'sebelumnya', 'banyak', 'yang', 'menjegal', 'amp', 'menggapai', 'sepele', 'amp', 'tidak', 'takut', 'dengan', 'suara', 'umat', 'islam', 'kita', 'buktikan', 'nanti', 'insya', 'allah', 'gantipres']
abusive_remove: ['kalau', 'dia', 'butuh', 'baik', 'nya', 'kayak', 'malaikat', 'yang', 'menempel', 'kayak', 'ulat', 'jambu',

abusive_remove: ['pengguna', 'cara', 'serang', 'ahok', 'galak']
abusive_remove: ['misi', 'membangun', 'manusia', 'jawa', 'barat', 'yang', 'memuliakan', 'agama', 'sehat', 'dan', 'unggul', 'no', 'jabarasyik', 'asyikm', 'di']
abusive_remove: ['pengguna', 'bukan', 'humor', 'wkwk']
abusive_remove: ['pengguna', 'ini', 'gue', 'baru', 'suka', 'semoga', 'siapapun', 'calon wakil presiden', 'yang', 'akan', 'mendampingi', 'bapak', 'jokowi', 'tidak', 'akan', 'menjadi', 'celah', 'bagi', 'oposan', 'untuk', 'merusak', 'kebersamaan', 'koalisi', 'pengusung', 'joko', 'widodo', 'dan', 'semoga', 'mampu', 'mendongkrak', 'suara', 'tentunya']
abusive_remove: []
abusive_remove: ['sumpah', 'menyesal', 'kayak', 'gue', 'banget', 'percaya', 'sama', 'dia', 'cihhh']
abusive_remove: ['pribumi', 'iseng', 'aparat', 'sangarrrr', 'puluhan cina', 'racist', 'aparat', 'kok', 'jongos']
abusive_remove: ['ketika', 'cinta', 'itu']
abusive_remove: ['pengguna', 'pengguna', 'dulu', 'jaman', 'sampai', 'negeri', 'di', 'boyolali', 'j

abusive_remove: ['pengguna', 'pengguna', 'dirtbag dan show', 'kalau', 'menurut', 'saya', 'film', 'yang', 'berkesan', 'yaitu', 'film', 'sih', 'dari', 'gua', 'hantu', 'karena', 'film', 'itu', 'mengindikasikan']
abusive_remove: ['kemarin', 'saya', 'dan', 'keluarga', 'pilih', 'demokrat', 'saya', 'sekeluarga', 'mantap', 'akan', 'pilih', 'gerindra']
abusive_remove: ['ngeri', 'banget', 'itu', 'hati', 'hati', 'nek', 'berdoa', 'semoga', 'depok', 'aman', 'semoga', 'saja', 'isinya', 'bukan', 'bom', 'benaran', 'semoga', 'tidak', 'ada', 'apa', 'apa', 'nya', 'allah', 'bulan', 'puasa', 'begini', 'bikin', 'tidak', 'tenang', 'saja', 'sih', 'ya']
abusive_remove: ['caplang', 'panci', 'kamu']
abusive_remove: ['pengguna', 'pengguna', 'pengguna', 'pengguna', 'pengguna', 'pengguna', 'pengguna', 'pengguna', 'begitulah', 'model', 'orang', 'yang', 'menukar', 'kebenaran', 'dengan', 'kesesatan', 'tuli', 'dan', 'melihat', 'kebenaran', 'maha', 'benar', 'allah', 'dengan', 'segala', 'firma']
abusive_remove: ['hai', '

abusive_remove: ['nostalgia', 'dengan', 'budaya', 'nos', 'tal', 'gia', 'dan', 'kerinduan', 'kadang', 'kadang', 'berlebihan', 'pada', 'sesuatu', 'yang', 'sangat', 'jauh', 'letaknya', 'atau', 'yang', 'sudah', 'tidak', 'ada', 'sekarang', 'kangen', 'sama', 'masa', 'masa', 'sekolah', 'kangen', 'sama', 'lingkungan', 'sekolah', 'yuk', 'nostalgia', 'masa', 'sekolah', 'dengan', 'pak']
abusive_remove: ['itu', 'orang', 'terbuat', 'dari', 'apa', 'bikin', 'gue', 'berkata', 'kasar', 'mulu']
abusive_remove: ['kepala kepolisian daerah', 'biadab', 'dan', 'tidak', 'punya', 'otak', 'kali']
abusive_remove: ['orang', 'yahudi', 'kristen', 'dan', 'muslim', 'yang', 'belum', 'pernah', 'bertemu', 'sebelumnya', 'berkumpul', 'tengah', 'malam', 'tepat', 'setelah', 'hari', 'terakhir', 'bulan', 'ramadan', 'untuk', 'menyanyikan', 'satu', 'lagu', 'dalam', 'tiga', 'bahasa', 'dan', 'dalam', 'tiga', 'bagian', 'harmoni', 'vokal', 'kami pilih rahmah']
abusive_remove: ['jangan', 'asal', 'berbicara', 'ndasmu', 'kamu', 'yang'

127.0.0.1 - - [01/Dec/2023 02:03:24] "POST /text-processing-file HTTP/1.1" 200 -


In [42]:
data = pd.read_sql_query('''select * from cleaningtext limit 10''', conn)
data

,cleaned_text
0,di saat semua cowok berusaha melacak perhatian...
1,pengguna pengguna siapa yang telat memberi tau...
2,kadang aku berpikir kenapa aku tetap percaya p...
3,pengguna aku itu aku dan ku tau matamu tapi di...
4,pengguna kaum sudah kelihatan dongoknya dari a...
5,ya dan kawan kawan
6,deklarasi pilihan kepala daerah aman dan anti ...
7,gue baru saja selesai re watch aldnoah zero pa...
8,nah admin belanja satu lagi port terbaik nak m...
9,enak lagi kalau sambil
